# Testing of ADA beer review dataset

In [3]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
WORK_DIR = os.getcwd()
DATA_DIR = "C:\\Users\\asus\\Desktop\\EPFL\\Course\\2023 Fall\\Applied data analysis\\Project\\Data"

## BeerAdvocate Dataset

In [5]:
DATA_BeerAdvocate = os.path.join(DATA_DIR, "BeerAdvocate")

### beers.csv

In [11]:
# read in the beers.csv file as a DataFrame
beers = pd.read_csv(os.path.join(DATA_BeerAdvocate, "beers.csv"))
beers.head()

,beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,nbr_reviews,avg,ba_score,bros_score,abv,avg_computed,zscore,nbr_matched_valid_ratings,avg_matched_valid_ratings
0,166064,Nashe Moskovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.7,NaN,NaN,0,NaN
1,166065,Nashe Pivovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,3.8,NaN,NaN,0,NaN
2,166066,Nashe Shakhterskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.8,NaN,NaN,0,NaN
3,166067,Nashe Zhigulevskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.0,NaN,NaN,0,NaN
4,166063,Zhivoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.5,NaN,NaN,0,NaN


### breweries.csv

In [12]:
breweries = pd.read_csv(os.path.join(DATA_BeerAdvocate, "breweries.csv"))
breweries.head()

,id,location,name,nbr_beers
0,39912,Kyrgyzstan,Abdysh-Ata (Абдыш Ата),5
1,39913,Kyrgyzstan,Arpa (АРПА),1
2,39914,Kyrgyzstan,Bear Beer,4
3,39915,Kyrgyzstan,Blonder Pub,4
4,39916,Kyrgyzstan,Kellers Bier,2


### users.csv

In [10]:
users = pd.read_csv(os.path.join(DATA_BeerAdvocate, "users.csv"))
users.head()

,nbr_ratings,nbr_reviews,user_id,user_name,joined,location
0,7820,465,nmann08.184925,nmann08,1.199704e+09,"United States, Washington"
1,2521,2504,stjamesgate.163714,StJamesGate,1.191838e+09,"United States, New York"
2,1797,1143,mdagnew.19527,mdagnew,1.116410e+09,Northern Ireland
3,31,31,helloloser12345.10867,helloloser12345,1.101380e+09,Northern Ireland
4,604,604,cypressbob.3708,cypressbob,1.069326e+09,Northern Ireland


### ratings.txt

In [6]:
# read in the file line by line
# extract the data into dataframes

# create the dataframe
def open_big_txt(folder, file_name):
    ratings = pd.DataFrame(columns = ["beer_name", "beer_id", "brewery_name", "brewery_id", "style", "abv", "date", "user_name", "user_id", "appearance", "aroma", "palate", "taste", "overall", "rating", "text", "review"])
    # open the file
    with open(os.path.join(folder, file_name+".txt"), "r") as f:
        # read the file line by line
        row = dict()
        for line in f:
            # if line number is larger than 1000, break
            if len(ratings) > 100:
                break
            # the text file recorded chunks of data representing each review
            # beer_name, beer_id, brewery_name, brewery_id, style, abv, date, user_name, user_id, appearance, aroma, palate, taste, overall, rating, text, review
            # the data is recorded in the order above, line by line iteratively
            
            # read in the data, split each line by the colon, and put them into a dataframe
            # represent each review as a row
            
            # split the line by the colon
            line = line.split(":")
            # if the line is empty, skip
            if len(line) < 2:
                continue
            else:
                if line[0] == "beer_name":
                    # start a new row
                    row = dict()
                    row[line[0]] = line[1].strip()
                elif line[0] == "review":
                    # end of the review
                    row[line[0]] = line[1].strip()
                    # append the row to the dataframe
                    ratings.loc[len(ratings)] = row
                else:
                    row[line[0]] = line[1].strip()        
    return ratings
        

In [16]:
ratings = open_big_txt(DATA_BeerAdvocate, "ratings")
ratings.head()

,beer_name,beer_id,brewery_name,brewery_id,style,abv,date,user_name,user_id,appearance,aroma,palate,taste,overall,rating,text,review
0,Régab,142544,Societe des Brasseries du Gabon (SOBRAGA),37262,Euro Pale Lager,4.5,1440064800,nmann08,nmann08.184925,3.25,2.75,3.25,2.75,3.0,2.88,"From a bottle, pours a piss yellow color with ...",True
1,Barelegs Brew,19590,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.5,1235127600,StJamesGate,stjamesgate.163714,3.0,3.5,3.5,4.0,3.5,3.67,Pours pale copper with a thin head that quickl...,True
2,Barelegs Brew,19590,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.5,1142247600,mdagnew,mdagnew.19527,4.0,3.5,3.5,4.0,3.5,3.73,"500ml Bottle bought from The Vintage, Antrim.....",True
3,Barelegs Brew,19590,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.5,1101898800,helloloser12345,helloloser12345.10867,4.0,3.5,4.0,4.0,4.5,3.98,Serving,True
4,Barelegs Brew,19590,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.5,1093860000,cypressbob,cypressbob.3708,4.0,4.0,4.0,4.0,4.0,4.00,"500ml bottlePours with a light, slightly hazy ...",True


## Matched beer dataset

In [12]:
DATA_MatchedBeers = os.path.join(DATA_DIR, "matched_beer_data")

In [13]:
# read in ratings_ba.txt file line by line and print out the first 100 lines
with open(os.path.join(DATA_MatchedBeers, "ratings_ba.txt"), "r") as f:
    for i, line in enumerate(f):
        if i > 100:
            break
        print(line)


beer_name: Legbiter

beer_id: 19827

brewery_name: Strangford Lough Brewing Company Ltd

brewery_id: 10093

style: English Pale Ale

abv: 4.8

date: 1417431600

user_name: Hellpop65

user_id: hellpop65.48993

appearance: nan

aroma: nan

palate: nan

taste: nan

overall: nan

rating: 3.25

text: nan

review: False



beer_name: Legbiter

beer_id: 19827

brewery_name: Strangford Lough Brewing Company Ltd

brewery_id: 10093

style: English Pale Ale

abv: 4.8

date: 1401357600

user_name: Latarnik

user_id: latarnik.52897

appearance: nan

aroma: nan

palate: nan

taste: nan

overall: nan

rating: 3.50

text: nan

review: False



beer_name: Legbiter

beer_id: 19827

brewery_name: Strangford Lough Brewing Company Ltd

brewery_id: 10093

style: English Pale Ale

abv: 4.8

date: 1393412400

user_name: RochefortChris

user_id: rochefortchris.697017

appearance: nan

aroma: nan

palate: nan

taste: nan

overall: nan

rating: 3.50

text: nan

review: False



beer_name: Legbiter

beer_id: 1982